# Generating bars out of trades

In [12]:
import pandas as pd
import numpy as np
import datetime as dt
import random as rd
import math
import quasardb
import quasardb.pandas as qdbpd
import plotly.graph_objects as go
from tqdm import trange, tqdm_notebook

In [13]:
#some parameters to seed data
#number of trades to generate
number_of_trades=100000
#how are they distributed in time, 5 -> one trade in each bin of 5s
freq_trades=2
#seed date
date = dt.datetime (2017, 3, 14, 9, 30, 0, 0)

In [14]:
#generates number_of_trades bins with intervals of size freq_trades 
index = pd.date_range(date, periods=number_of_trades, freq=str(freq_trades)+'s')
#create randonmness for trade in each bin
jitter=np.random.randint(0, freq_trades-1,number_of_trades )
jitter_timedelta=pd.to_timedelta(jitter,unit='s')
#index for number_of_trades happening randomly in bins of size freq_trades
index= index+jitter_timedelta
#create using index trades with an arbitrary price in range [150,155]
trades= pd.DataFrame (
    data={'price':np.round(np.random.uniform(
            low=150.0,
            high=155.0,
            size=(number_of_trades,)),2),
        'volume':np.random.randint(10, 110,number_of_trades )
         }
    ,index=index
)

In [15]:
#connect to QDB and insert trades
c = quasardb.Cluster("qdb://qdb-server:2836")


In [16]:
#create table for this sample
table = c.ts("tradesohlc")

columns = [quasardb.ColumnInfo(quasardb.ColumnType.Double, "price"),
           quasardb.ColumnInfo(quasardb.ColumnType.Int64, "volume")]
table.create(columns)


In [17]:
#insert using batch inserter
#inserting using python is not optimal in QDQ
sample_size = 10000
batch_columns = [quasardb.BatchColumnInfo("tradesohlc", "price", sample_size),
                 quasardb.BatchColumnInfo("tradesohlc", "volume", sample_size)]
batch_inserter = c.ts_batch(batch_columns)

for row in tqdm_notebook(range(len(trades.index)), desc='Inserting in QDB'):
#for row in range(len(trades.index)):
    batch_inserter.start_row(np.datetime64(trades.index[row], 'ns'))
    batch_inserter.set_double(0, trades.price.iloc[row]) # set close
    batch_inserter.set_int64(1, trades.volume.iloc[row]) # set volume
    if ( not row % (sample_size-1)):
        batch_inserter.push()

In [18]:
#after all this hard work we can now create bars 
#here bars of 30s
query= c.query("select first(price) , max(price), min(price)  , last( price ) , sum(volume )\
                from tradesohlc in range ( 2017-03-14 ,2017-03-15 ) group by 1min ")
query_result = query.run()


In [19]:
#convert result as a pandas dataframe
tmp = dict()
for col in query_result.tables['tradesohlc']:
    tmp[col.name] = col.data
    result_as_pandas = pd.DataFrame.from_dict(tmp, orient='columns').set_index('$timestamp')
result_as_pandas.shape

(870, 5)

In [20]:
result_as_pandas[:3]

,first(price),max(price),min(price),last(price),sum(volume)
$timestamp,,,,,
2017-03-14 09:30:00,152.47,154.86,150.05,154.06,1577
2017-03-14 09:31:00,151.35,154.98,150.04,153.63,1552
2017-03-14 09:32:00,153.04,154.92,150.39,151.07,1940


In [21]:
fig = go.Figure(data=[go.Candlestick(x=result_as_pandas.index,
                open=result_as_pandas['first(price)'],
                high=result_as_pandas['max(price)'],
                low=result_as_pandas['min(price)'],
                close=result_as_pandas['last(price)'])])

fig.show()

In [11]:
#tearing down data for this sample
table = c.ts("tradesohlc")
table.remove()